<a href="https://colab.research.google.com/github/chunkywizard1992/Notesforclasss/blob/main/Copy_of_quick_tour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick Tour

<a target="_blank" href="https://colab.research.google.com/github/lavague-ai/LaVague/blob/main/docs/docs/get-started/quick-tour-notebook/quick-tour.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

## Pre-requisites

**Note**: We use OpenAI's models, for the embedding, LLM and Vision model. You will need to set the OPENAI_API_KEY variable in your local environment with a valid API key for this example to work.

If you don't have an OpenAI API key, please get one here: https://platform.openai.com/docs/quickstart/developer-quickstart

# Installation

We start by download LaVague.

In [218]:
!pip install flask google-auth google-auth-oauthlib google-auth-httplib2 pyngrok


In [219]:
import os
from pyngrok import ngrok

# Set up an ngrok tunnel to your local server
public_url = ngrok.connect(5000)
print("Ngrok URL:", public_url)

ERROR:pyngrok.process.ngrok:t=2024-05-26T01:13:25+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-05-26T01:13:25+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

We will need to set our OpenAI Key as a Colab secret (see the key icon on the left-hand side of the Colab notebook) named 'OPENAI_API_KEY' and then convert it to an environment variable with the same name.

In [217]:
import os
from flask import Flask, redirect, url_for, session, request
from google.oauth2 import id_token
from google_auth_oauthlib.flow import Flow
import google.auth.transport.requests

# Configure the OAuth 2.0 client
CLIENT_ID = '1070926917397-hpren3rf4p2nc65aoc6s6hgdllqmiqk6.apps.googleusercontent.com'
CLIENT_SECRET = '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'
REDIRECT_URI = public_url + '/callback'  # Use the public ngrok URL

SCOPES = ['https://www.googleapis.com/auth/userinfo.profile', 'https://www.googleapis.com/auth/userinfo.email']

flow = Flow.from_client_config(
    {
        "web": {
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET,
            "redirect_uris": [REDIRECT_URI],
            "auth_uri": "https://accounts.google.com/o/oauth2/auth",
            "token_uri": "https://oauth2.googleapis.com/token"
        }
    },
    scopes=SCOPES
)

# Initialize Flask app
app = Flask(__name__)
app.secret_key = os.urandom(24)

@app.route('/')
def index():
    return 'Welcome to the Google OAuth 2.0 Colab example! <a href="/login">Login</a>'

@app.route('/login')
def login():
    authorization_url, state = flow.authorization_url()
    session['state'] = state
    return redirect(authorization_url)

@app.route('/callback')
def callback():
    flow.fetch_token(authorization_response=request.url)

    if not session['state'] == request.args['state']:
        return redirect(url_for('index'))

    credentials = flow.credentials
    request_session = google.auth.transport.requests.Request()

    id_info = id_token.verify_oauth2_token(
        id_token=credentials.id_token,
        request=request_session,
        audience=CLIENT_ID
    )

    session['user_info'] = id_info
    return redirect(url_for('profile'))

@app.route('/profile')
def profile():
    user_info = session.get('user_info')
    if user_info:
        return f"Logged in as {user_info['name']} ({user_info['email']})"
    return redirect(url_for('index'))

# Run the app
if __name__ == '__main__':
    app.run('localhost', 5000)



NameError: name 'public_url' is not defined

In [ ]:
get_ipython().system_raw('python3 -m flask run --host=0.0.0.0 --port=5000 &')


In [196]:
# Cell 3: Set OpenAI API Key
import os

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
else:
    os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [197]:
# Cell 4: Import necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import os

In [198]:
# Cell 5: Initialize the Selenium WebDriver
def init_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--disable-translate")
    chrome_options.add_argument("--disable-software-rasterizer")
    chrome_options.binary_location = "/usr/bin/google-chrome"

    driver = webdriver.Chrome(service=Service("/usr/local/bin/chromedriver"), options=chrome_options)
    return driver

In [199]:
# Cell 6: Define the function to log into Google
def login_to_google(driver, email, password, retries=10):
    for i in range(retries):
        try:
            driver.get("https://accounts.google.com/")
            break
        except Exception as e:
            print(f"Connection failed: {e}")
            if i == retries - 1:
                raise e

    try:
        # Enter email
        email_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.NAME, "identifier"))
        )
        email_input.send_keys(email)
        email_input.send_keys(Keys.RETURN)

        # Wait for password input to appear and enter password
        password_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.NAME, "password"))
        )
        password_input.send_keys(password)
        password_input.send_keys(Keys.RETURN)

        # Wait for login to complete
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='myaccount.google.com']"))
        )
        print("Logged into Google successfully.")
    except TimeoutException as e:
        print(f"TimeoutException: {e}")
    finally:
        driver.quit()




## ActionEngine

**An WebAgent is made up of two components: an `ActionEngine` and a `WorldModel`.**

Let's start by initializing an `ActionEngine`, which is responsible for generating automation code for text instructions and executing them.

In [200]:
# Cell 7: Initialize ActionEngine, PythonEngine, and WorldModel
from lavague.core import ActionEngine
from lavague.drivers.selenium import SeleniumDriver
from lavague.core import PythonEngine, WorldModel

selenium_driver = SeleniumDriver()
action_engine = ActionEngine(selenium_driver)
python_engine = PythonEngine()
world_model = WorldModel()

# Python Engine

The python engine is responsible for generating code that doesn't interact with an html page.

In [201]:
# Cell 8: Initialize WebAgent
from lavague.core.agents import WebAgent

agent = WebAgent(world_model, action_engine, python_engine)


In [208]:
# Cell 9: Initialize ActionEngine, PythonEngine, and WorldModel
from lavague.core import ActionEngine, PythonEngine, WorldModel
from lavague.drivers.selenium import SeleniumDriver

def init_driver():
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
    from webdriver_manager.chrome import ChromeDriverManager

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--disable-translate")
    chrome_options.binary_location = "/usr/bin/google-chrome"

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def login_to_google(driver, email, password, retries=10):
    for i in range(retries):
        try:
            driver.get("https://accounts.google.com/")
            break
        except Exception as e:
            print(f"Connection failed: {e}")
            if i == retries - 1:
                raise e

    try:
        print("Attempting to find email input...")
        email_input = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.NAME, "identifier"))
        )
        print("Email input found, entering email...")
        email_input.send_keys(email)
        email_input.send_keys(Keys.RETURN)

        # Check for any intermediate steps
        try:
            print("Checking for intermediate steps...")
            next_button = WebDriverWait(driver, 40).until(
                EC.element_to_be_clickable((By.XPATH, '//span[text()="Next"]'))
            )
            next_button.click()
        except TimeoutException:
            pass  # No intermediate step

        # Another possible "Next" button
        try:
            next_button_alt = WebDriverWait(driver, 40).until(
                EC.element_to_be_clickable((By.XPATH, '//div[@id="identifierNext"]//button'))
            )
            next_button_alt.click()
        except TimeoutException:
            pass  # No intermediate step

        print("Waiting for password input to appear...")
        password_input = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.NAME, "password"))
        )
        print("Password input found, entering password...")
        password_input.send_keys(password)
        password_input.send_keys(Keys.RETURN)

        print("Waiting for login to complete...")
        WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='myaccount.google.com']"))
        )
        print("Logged into Google successfully.")
    except TimeoutException as e:
        print(f"TimeoutException: {e}")
        driver.save_screenshot("error_screenshot.png")
        page_source = driver.page_source
        with open("page_source.html", "w") as f:
            f.write(page_source)
        print("Screenshot and page source saved for analysis.")
    finally:
        driver.quit()

def login_to_gmail_with_agent(agent, email, password):
    selenium_driver.driver = init_driver()
    login_to_google(selenium_driver.driver, email, password)

# Example usage
email = "bradmatera@gmail.com"
password = os.getenv('your-password')  # Assuming you set this secret in Colab
login_to_gmail_with_agent(agent, email, password)



Attempting to find email input...
Email input found, entering email...
Checking for intermediate steps...
Waiting for password input to appear...
TimeoutException: Message: 
Stacktrace:
#0 0x5a4a11313e8a <unknown>
#1 0x5a4a10ffd45c <unknown>
#2 0x5a4a110495b5 <unknown>
#3 0x5a4a11049671 <unknown>
#4 0x5a4a1108df14 <unknown>
#5 0x5a4a1106c4dd <unknown>
#6 0x5a4a1108b2cc <unknown>
#7 0x5a4a1106c253 <unknown>
#8 0x5a4a1103c1c7 <unknown>
#9 0x5a4a1103cb3e <unknown>
#10 0x5a4a112da2cb <unknown>
#11 0x5a4a112de377 <unknown>
#12 0x5a4a112c6dfe <unknown>
#13 0x5a4a112dee42 <unknown>
#14 0x5a4a112ab79f <unknown>
#15 0x5a4a11303178 <unknown>
#16 0x5a4a1130334b <unknown>
#17 0x5a4a11312fbc <unknown>
#18 0x7e10e7630ac3 <unknown>

Screenshot and page source saved for analysis.
